In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Loading environment variables from .env file
load_dotenv()

True

In [2]:
# Getting the Steam API key from the environment variable
api_key = os.getenv('STEAM_API_KEY')
base_url = 'https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/'


In [3]:
def get_player_count(appid):
    # Define the parameters for the API request, including the appid and the API key
    params = {
        'appid': appid,
        'key': api_key
    }
    
    # Send a GET request to the Steam API with the specified parameters
    response = requests.get(base_url, params=params)
    
    # Parse the JSON response from the API
    data = response.json()
    
    # Check if the response contains the 'player_count' key within the 'response' key
    if 'response' in data and 'player_count' in data['response']:
        # If 'player_count' exists, return its value
        return data['response']['player_count']
    else:
        # If 'player_count' does not exist, return None
        return None


In [4]:
# Define the app IDs for the games you are interested in
games = {
    'The Isle': 376210,
    'ARK: Survival Evolved': 346110,
    'No Man\'s Sky': 275850
}

# Fetch player counts
player_counts = {}
for game, appid in games.items():
    player_counts[game] = get_player_count(appid)

# Display the player counts
player_counts

{'The Isle': 3234, 'ARK: Survival Evolved': 19090, "No Man's Sky": 27509}

In [7]:
def get_global_stats_for_game(appid, stat_name, startdate, enddate):
    url = 'https://api.steampowered.com/ISteamUserStats/GetGlobalStatsForGame/v1/'
    params = {
        'appid': appid,
        'count': 1,
        'name[0]': stat_name,
        'startdate': startdate,
        'enddate': enddate,
        'key': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    print(f"Response for appid {appid}: {data}")  # Print the response for debugging
    
    if 'response' in data and 'result' in data['response']:
        return data['response']['result']
    else:
        return None

In [8]:
# Define the stat name and date range for the global stats
stat_name = 'player_count'  # Adjust the stat name if needed
startdate = int((pd.Timestamp.now() - pd.Timedelta(days=30)).timestamp())  # Example: last 30 days
enddate = int(pd.Timestamp.now().timestamp())

# Fetch global stats for each game
global_stats = {}
for game, appid in games.items():
    stats = get_global_stats_for_game(appid, stat_name, startdate, enddate)
    global_stats[game] = stats

# Display the global stats for each game
for game, stats in global_stats.items():
    print(f"{game}: {stats}")

Response for appid 376210: {'response': {'result': 8, 'error': "Stat 'player_count' not found"}}
Response for appid 346110: {'response': {'result': 8, 'error': "Stat 'player_count' not found"}}
Response for appid 275850: {'response': {'result': 8, 'error': "Stat 'player_count' not found"}}
The Isle: 8
ARK: Survival Evolved: 8
No Man's Sky: 8
